In [1]:
#IMPORT PACKAGES
from rdflib import Graph, URIRef
import os.path
import importlib
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
import ifcopenshell.util.selector
import random as rd
import pandas as pd
from tabulate import tabulate
import cv2


#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%autoreload 2

UsageError: Line magic function `%autoreload` not found.


In [3]:
%load_ext autoreload

In [4]:
# path='D:\geomapi\tests\testfiles'
from pathlib import Path
import pye57
import laspy

#PATH
path= Path('D:\geomapi') / "tests" / "testfiles"  

#RESOURCES
resourceGraphPath=path /  'graphs' / 'parking_resource_graph.ttl'
resourceGraph=Graph().parse(str(resourceGraphPath))
print(f'loaded {resourceGraphPath}')

#POINTCLOUD
pcdPath=path / 'pcd'/"parking.pcd"
pcd=o3d.io.read_point_cloud(str(pcdPath))
print(f'loaded {pcd}')

e57Path1=path / 'pcd'/"lidar.e57"
xmlPath=path / 'pcd'/"lidar.xml"
e57_1 = pye57.E57( str(e57Path1))
e571=e57_1.read_scan_raw(1) 
print(f'loaded {e57Path1}')

e57Path2=path / 'pcd'/"parking.e57"
e57_2 = pye57.E57( str(e57Path2))
e572=e57_2.read_scan_raw(0) 
print(f'loaded {e57Path2}')

lasPath=path / 'pcd'/"parking.las"
las=laspy.read(str(lasPath))
print(f'loaded {lasPath}')

pcdGraphpath=path / 'graphs' /  'pcd_graph.ttl'
pcdGraph=Graph().parse(pcdGraphpath)
print(f'loaded {pcdGraphpath}')           


#MESH
meshPath=path / 'mesh'/ 'parking.obj'
mesh=o3d.io.read_triangle_mesh(str(meshPath))
print(f'loaded {mesh}') 

meshGraphPath=path / 'graphs' /  'mesh_graph.ttl'
meshGraph=Graph().parse(str(meshGraphPath))
print(f'loaded {meshGraphPath}')           


#IFC
ifcPath=path / 'ifc' / "parking.ifc"
bimNodes=tl.ifc_to_nodes_multiprocessing(str(ifcPath)) #! Note: this uses geomapi functionality
print(f'loaded {len(bimNodes)} bimNodes from ifc file')

ifcGraphPath=path /  'graphs' / 'parking_ifc_graph.ttl'
ifcGraph=Graph().parse(str(ifcGraphPath))
print(f'loaded {ifcGraphPath}')           

#IMG
csvPath=path / 'IMG' / 'parking.csv' #! we don't do anything with the csv
imgGraphPath=path /  'graphs' / 'road_img_graph.ttl'
imgGraph=Graph().parse(str(imgGraphPath))
print(f'loaded {imgGraphPath}')    

imageXmpPath1 = path / 'IMG' / 'DJI_0085.xmp'
imagePath1=path / 'IMG' / "DJI_0085.JPG" 
image1=cv2.imread(str(imagePath1))
imageCartesianTransform1= np.array([[-8.13902571e-02,  6.83059476e-01 ,-7.25813597e-01,  5.18276221e+01],
                                    [ 9.96648497e-01,  4.97790854e-02, -6.49139139e-02 , 6.10007435e+01],
                                    [-8.20972697e-03, -7.28664391e-01, -6.84821733e-01,  1.50408221e+01],
                                    [ 0.00000000e+00 , 0.00000000e+00, 0.00000000e+00 , 1.00000000e+00]])
print(f'loaded {imagePath1}')           

imageXmpPath2 = path / 'IMG' / 'IMG_8834.xmp'
imagePath2=path / 'IMG' / "IMG_8834.JPG" 
image2=cv2.imread(str(imagePath2))
imageCartesianTransform2= np.array([[ 4.12555151e-01,  4.12058430e-02 ,-9.10000179e-01, 6.68850552e+01],
                                    [ 9.10841440e-01, -4.52553581e-03,  4.12731621e-01 , 4.52551195e+01],
                                    [ 1.28887160e-02 ,-9.99140430e-01 ,-3.93990225e-02 , 5.45377093e+00],
                                    [ 0.00000000e+00 , 0.00000000e+00  ,0.00000000e+00 , 1.00000000e+00]])
print(f'loaded {imagePath2}')    


loaded D:\geomapi\tests\testfiles\graphs\parking_resource_graph.ttl
loaded PointCloud with 556485 points.
loaded D:\geomapi\tests\testfiles\pcd\lidar.e57
loaded D:\geomapi\tests\testfiles\pcd\parking.e57
loaded D:\geomapi\tests\testfiles\pcd\parking.las
loaded D:\geomapi\tests\testfiles\graphs\pcd_graph.ttl
loaded TriangleMesh with 62143 points and 49999 triangles.
loaded D:\geomapi\tests\testfiles\graphs\mesh_graph.ttl
loaded 2368 bimNodes from ifc file
loaded D:\geomapi\tests\testfiles\graphs\parking_ifc_graph.ttl
loaded D:\geomapi\tests\testfiles\graphs\road_img_graph.ttl
loaded D:\geomapi\tests\testfiles\IMG\DJI_0085.JPG
loaded D:\geomapi\tests\testfiles\IMG\IMG_8834.JPG


In [5]:
# imgNodes=[geomapi.imgNode(xmlPath=imageXmpPath1),imgNode(xmlPath=imageXmpPath1)]
imgNode1=ImageNode(xmpPath=imageXmpPath1,getResource=True)
imgNode2=ImageNode(xmpPath=imageXmpPath2,getResource=True)
meshNode=MeshNode(path=meshPath,getResource=True)
pcdNode=PointCloudNode(path=pcdPath,getResource=True)

In [9]:
#get list of files
files=[str(path / 'IMG' / 'DJI_0085.xmp'),str(path / 'IMG' / 'IMG_8834.xmp')]
print(files)
#select every fifth files
imgNodes=[]
for f in files:     
    n=ImageNode(name=ut.get_filename(f),  #! this took quite long from remote drive                            
                                xmpPath=f,getResource=True)
    del n.resource
    if getattr(n,'imageWidth',None) is not None:
    #delete resource
    

        #set focallength and cartesianTransform
        n.focalLength35mm=n.focalLength35mm/36*n.imageWidth #! multiple definitions possible    
        rotation= n.cartesianTransform[0:3,0:3].T #! RC uses column-based rotaton matrix
        translation=n.cartesianTransform[0:3,3]
        n.cartesianTransform=gmu.get_cartesian_transform(rotation=rotation,translation=translation)
        
        imgNodes.append(n)
    
number=len(imgNodes)
imgNodes=[n for n in imgNodes if all(n.cartesianTransform[0:3,3] !=0)]
joinedImages=gmu.join_geometries([gmu.generate_visual_cone_from_image(n.cartesianTransform, height =1).paint_uniform_color([1,0,0]) for n in imgNodes])
print(f"{len(imgNodes)} / {number} Nodes created (remainder is not properly localised)!")

['D:\\geomapi\\tests\\testfiles\\IMG\\DJI_0085.xmp', 'D:\\geomapi\\tests\\testfiles\\IMG\\IMG_8834.xmp']
2 / 2 Nodes created (remainder is not properly localised)!


In [8]:
tl.nodes_to_graph(imgNodes,graphPath=str(path / 'img' / 'parking_img_graph.ttl'),save=   True)

<Graph identifier=N45ae4ee814a9438c9dbf82f0120ea5f3 (<class 'rdflib.graph.Graph'>)>

In [7]:
ut.cartesianTransform_to_literal(imageCartesianTransform1)

'[[-8.13902571e-02]\n [ 6.83059476e-01]\n [-7.25813597e-01]\n [ 5.18276221e+01]\n [ 9.96648497e-01]\n [ 4.97790854e-02]\n [-6.49139139e-02]\n [ 6.10007435e+01]\n [-8.20972697e-03]\n [-7.28664391e-01]\n [-6.84821733e-01]\n [ 1.50408221e+01]\n [ 0.00000000e+00]\n [ 0.00000000e+00]\n [ 0.00000000e+00]\n [ 1.00000000e+00]]'

In [10]:
assert '-8.1390571e-02' in ut.cartesianTransform_to_literal(imageCartesianTransform1)

AssertionError: 